In [1]:
import pandas as pd
import re
from sklearn.utils import shuffle
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification,classification_report,accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
lemaoi = WordNetLemmatizer()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [190]:
raw_data = pd.read_csv('train_data.csv')
raw_data

,name,brand,categories,primaryCategories,reviews.date,reviews.text,reviews.title,sentiment
0,"All-New Fire HD 8 Tablet, 8"" HD Display, Wi-Fi...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Electronics,2016-12-26T00:00:00.000Z,Purchased on Black FridayPros - Great Price (e...,Powerful tablet,Positive
1,Amazon - Echo Plus w/ Built-In Hub - Silver,Amazon,"Amazon Echo,Smart Home,Networking,Home & Tools...","Electronics,Hardware",2018-01-17T00:00:00.000Z,I purchased two Amazon in Echo Plus and two do...,Amazon Echo Plus AWESOME,Positive
2,Amazon Echo Show Alexa-enabled Bluetooth Speak...,Amazon,"Amazon Echo,Virtual Assistant Speakers,Electro...","Electronics,Hardware",2017-12-20T00:00:00.000Z,Just an average Alexa option. Does show a few ...,Average,Neutral
3,"Fire HD 10 Tablet, 10.1 HD Display, Wi-Fi, 16 ...",Amazon,"eBook Readers,Fire Tablets,Electronics Feature...","Office Supplies,Electronics",2017-08-04T00:00:00.000Z,"very good product. Exactly what I wanted, and ...",Greattttttt,Positive
4,"Brand New Amazon Kindle Fire 16gb 7"" Ips Displ...",Amazon,"Computers/Tablets & Networking,Tablets & eBook...",Electronics,2017-01-23T00:00:00.000Z,This is the 3rd one I've purchased. I've bough...,Very durable!,Positive
...,...,...,...,...,...,...,...,...
3995,Amazon - Echo Plus w/ Built-In Hub - Silver,Amazon,"Amazon Echo,Smart Home,Networking,Home & Tools...","Electronics,Hardware",2017-12-08T00:00:00.000Z,"It‚Äôs fun for the family to play with, but it...",Fun toy,Positive
3996,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,2017-03-31T00:00:00.000Z,"I love the Kindle, it is a great product. It r...",Great Product,Positive
3997,Amazon Tap - Alexa-Enabled Portable Bluetooth ...,Amazon,"Amazon Echo,Home Theater & Audio,MP3 MP4 Playe...",Electronics,2017-01-19T00:00:00.000Z,I was looking for a blutooth speaker to use wi...,"Great ""dumb"" speaker",Positive
3998,"Brand New Amazon Kindle Fire 16gb 7"" Ips Displ...",Amazon,"Computers/Tablets & Networking,Tablets & eBook...",Electronics,2016-05-27T00:00:00.000Z,This is the second Amazon Fire 7 tablet I have...,A great 7 inch tablet at a great price,Positive


In [191]:
print(raw_data['sentiment'].value_counts())
print('It is very clear that the data is not balanced to make it balanced we will 100 sample')

Positive    3749
Neutral      158
Negative      93
Name: sentiment, dtype: int64
It is very clear that the data is not balanced to make it balanced we will 100 sample


In [211]:
raw_data = shuffle(raw_data)

In [137]:
def under_sample(raw):
    posi = raw_data.loc[raw_data['sentiment']=='Positive'][:100]
    Neu = raw_data.loc[raw_data['sentiment']=='Neutral'][:100]
    Nega = raw_data.loc[raw_data['sentiment']=='Negative']
    traning_data = pd.concat([posi, Neu, Nega])
    traning_data.reset_index(inplace=True)
    traning_data['sentiment'].replace({'Positive':1,'Negative':-1,'Neutral':0}, inplace=True)
    return traning_data

In [138]:
def over_sample(raw):
    posi = raw_data.loc[raw_data['sentiment']=='Positive'][:1000]
    Neu = raw_data.loc[raw_data['sentiment']=='Neutral']
    Nega = raw_data.loc[raw_data['sentiment']=='Negative']
    Neu_new = pd.concat([Neu,Neu,Neu,Neu,Neu,Neu], axis=0)
    Nega_new = pd.concat([Nega,Nega,Nega,Nega,Nega,Nega,Nega,Nega,Nega,Nega])
    traning_data = pd.concat([posi, Neu_new, Nega_new])
    traning_data.reset_index(inplace=True)
    traning_data['sentiment'].replace({'Positive':1,'Negative':-1,'Neutral':0}, inplace=True)
    return traning_data
    

In [212]:
train_frame = over_sample(raw_data)
train_frame.columns

Index(['index', 'name', 'brand', 'categories', 'primaryCategories',
       'reviews.date', 'reviews.text', 'reviews.title', 'sentiment'],
      dtype='object')

In [213]:
print('Clubbing Text and Title of Reviews of Amazon Dataset')
train_frame['data'] = train_frame['reviews.text'] + ' '+ train_frame['reviews.title']
data_to_train = train_frame.loc[:,['data','sentiment']]
data_to_train

Clubbing Text and Title of Reviews of Amazon Dataset


,data,sentiment
0,I purchased this on Black Friday because I cou...,1
1,"this was a christmas present, for the nephew w...",1
2,Setting up the amazon show was a little tricky...,1
3,Works great for my grandson - had the Geek squ...,1
4,Very impress with the echo show and all of its...,1
...,...,...
2873,I do not like this Tablet one bit. It has to b...,-1
2874,1st kindle screen failed & had to reboot often...,-1
2875,Returned product cause I did not like it and r...,-1
2876,The last 2 models of Kindle HDX 8 have been te...,-1


In [214]:
data_to_train.dropna(inplace=True, axis=0)

In [215]:
data_to_train.head(5)

,data,sentiment
0,I purchased this on Black Friday because I cou...,1
1,"this was a christmas present, for the nephew w...",1
2,Setting up the amazon show was a little tricky...,1
3,Works great for my grandson - had the Geek squ...,1
4,Very impress with the echo show and all of its...,1


In [216]:
def pre_processing(x):
    x = x.lower()
    x = re.sub('[^a-z0-9 ]','',x)
    result = []
    for i in x.split():
        if i in stopwords.words('english'):
            pass
        else:
            if len(i) <3:
                pass
            else:
                result.append(lemaoi.lemmatize(i))
    res = ' '.join(result)
    res = re.sub('\d{2,}\w+','',res)
    return res 

In [217]:
data_to_train['cleaned_data'] = data_to_train['data'].apply(lambda x: pre_processing(x))
data_to_train

,data,sentiment,cleaned_data
0,I purchased this on Black Friday because I cou...,1,purchased black friday could pas price tablet ...
1,"this was a christmas present, for the nephew w...",1,christmas present nephew work good love work good
2,Setting up the amazon show was a little tricky...,1,setting amazon show little trickybut managed n...
3,Works great for my grandson - had the Geek squ...,1,work great grandson geek squad get security pl...
4,Very impress with the echo show and all of its...,1,impress echo show feature love
...,...,...,...
2873,I do not like this Tablet one bit. It has to b...,-1,like tablet one bit registered amazon function...
2874,1st kindle screen failed & had to reboot often...,-1,1st kindle screen failed reboot often customer...
2875,Returned product cause I did not like it and r...,-1,returned product cause like really use returned
2876,The last 2 models of Kindle HDX 8 have been te...,-1,last model kindle hdx terrible purchased model...


In [218]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data_to_train['cleaned_data'])
feature_names = vectorizer.get_feature_names()
X = vectors.todense()
Y = data_to_train['sentiment']

In [200]:
pipeline_linear = make_pipeline(StandardScaler(),LogisticRegression(solver = 'lbfgs'))
pipeline_knn = make_pipeline(StandardScaler(),KNeighborsClassifier(n_neighbors=15))
pipeline_forest = make_pipeline(StandardScaler(),RandomForestClassifier(n_estimators=300))
pipeline_ada = make_pipeline(StandardScaler(),AdaBoostClassifier(n_estimators=150))
pipeline_gradient = make_pipeline(StandardScaler(),GradientBoostingClassifier(n_estimators=60,learning_rate=1))

In [219]:
x_train, x_test, y_train, y_test= train_test_split(X,Y)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(2157, 2828)
(719, 2828)
(2157,)
(719,)


In [220]:
file_obj = open('test_results.txt','w')
file_obj.writelines('Over Sample')
file_obj.writelines('\n')

In [221]:
for i in [pipeline_forest,pipeline_knn,pipeline_linear,pipeline_ada,pipeline_gradient]:
    file_obj.writelines(str(i))
    file_obj.writelines('\n')
    obj = i.fit(x_train,y_train)
    y_pred = obj.predict(x_test)
    file_obj.writelines(classification_report(y_test,y_pred))
    print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       1.00      1.00      1.00       253
           0       0.97      1.00      0.98       225
           1       1.00      0.97      0.98       241

    accuracy                           0.99       719
   macro avg       0.99      0.99      0.99       719
weighted avg       0.99      0.99      0.99       719

              precision    recall  f1-score   support

          -1       0.98      0.94      0.96       253
           0       0.88      0.97      0.93       225
           1       0.99      0.93      0.96       241

    accuracy                           0.95       719
   macro avg       0.95      0.95      0.95       719
weighted avg       0.95      0.95      0.95       719

              precision    recall  f1-score   support

          -1       0.99      1.00      0.99       253
           0       0.93      1.00      0.96       225
           1       1.00      0.91      0.95       241

    accuracy        

In [222]:
file_obj.close()

In [180]:
x_test = pd.read_csv('test_data.csv')
x_test['data'] = x_test['reviews.text'] + ' '+ x_test['reviews.title']
x_test.dropna(axis=0, inplace=True)

In [181]:
x_test['cleaned'] = x_test['data'].apply(lambda x :pre_processing(x))

In [182]:
vector = vectorizer.transform(x_test['cleaned'])
x_test_new = vector.todense()

In [183]:
y_pred_test = pipeline_forest.predict(x_test_new)

In [184]:
x_test['predicted_under'] = y_pred_test

In [185]:
x_test.to_excel('submiss_under.xlsx')